# Segmentatation
ref:
https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/how_to_use_nnunet.md
https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/set_environment_variables.md 


env: python3.12 torch

## Example dataset.json:
```json
{
    "channel_names": {
        "0": "CT"
    },
    "labels": {
        "background": 0,
        "blood": 1,
        "aaa": 2
    },
    "numTraining": 50,
    "file_ending": ".nii.gz"
}
```

In [1]:
%env nnUNet_raw=data/nnunet_raw
%env nnUNet_preprocessed=data/nnunet_preprocessed
%env nnUNet_results=data/nnunet_results

!nnUNetv2_plan_and_preprocess -d 2 --verify_dataset_integrity

env: nnUNet_raw=data/nnunet_raw
env: nnUNet_preprocessed=data/nnunet_preprocessed
env: nnUNet_results=data/nnunet_results
Fingerprint extraction...
Dataset002_perlin_aaa
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer
100%|███████████████████████████████████████████| 50/50 [00:05<00:00,  9.39it/s]
Experiment planning...

############################
INFO: You are using the old nnU-Net default planner. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Dropping 3d_lowres config because the image size difference to 3d_fullres is too small. 3d_fullres: [213. 130. 149.], 3d_l

In [16]:
!nnUNet_n_proc_DA=8 nnUNetv2_train 1 2d all


############################
INFO: You are using the old nnU-Net default plans. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Using device: cuda:0
/home/tommy/miniconda3/envs/next-synthseg/lib/python3.12/site-packages/nnunetv2/training/nnUNetTrainer/nnUNetTrainer.py:164: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.grad_scaler = GradScaler() if self.device.type == 'cuda' else None

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
###########################################

In [18]:
!nnUNetv2_find_best_configuration 1 -c 2d -f all

usage: nnUNetv2_find_best_configuration [-h] [-p P [P ...]] [-c C [C ...]]
                                        [-tr TR [TR ...]] [-np NP]
                                        [-f F [F ...]] [--disable_ensembling]
                                        [--no_overwrite]
                                        dataset_name_or_id
nnUNetv2_find_best_configuration: error: argument -f: invalid int value: 'all'


In [25]:
!nnUNetv2_predict -i "data/nnunet_raw/Dataset001_aaa/imagesTs" -o "data/nnunet_results/Dataset001_aaa/labelsTs" -d 1 -c 2d -f all


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

/home/tommy/miniconda3/envs/next-synthseg/lib/python3.12/site-packages/nnunetv2/inference/predict_from_raw_data.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be execute

In [2]:
image = nib.load(
    "data/nnunet_raw/Dataset001_aaa/imagesTs/aaa_000_0000.nii.gz"
).get_fdata()
label = nib.load(
    "data/nnunet_results/Dataset001_aaa/labelsTs/aaa_000.nii.gz"
).get_fdata()
ground_truth = nib.load(
    "data/nnunet_raw/Dataset001_aaa/labelsTs/aaa_000.nii.gz"
).get_fdata()
check_image(
    [
        {"image": label, "title": "label", "is_label": True},
        {"image": ground_truth, "title": "ground_truth", "is_label": True},
        {"image": image, "title": "image", "is_label": False},
    ]
)

interactive(children=(IntSlider(value=0, description='slice_index_0', max=125), IntSlider(value=0, description…

In [10]:
label.shape,ground_truth.shape

((110, 104, 126), (110, 104, 126))

In [11]:
import torch
import torch.nn.functional as F

def calculate_dice_score(pred, target):
    # Convert numpy arrays to torch tensors
    pred = torch.from_numpy(pred).float()
    target = torch.from_numpy(target).float()
    
    # Flatten the tensors
    pred = pred.flatten()
    target = target.flatten()
    
    # Calculate intersection and union
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum()
    
    # Calculate Dice score
    dice = (2. * intersection) / (union + 1e-8)  # Adding small epsilon to avoid division by zero
    
    return dice.item()

# Calculate Dice score for each class
num_classes = int(max(label.max(), ground_truth.max())) + 1
dice_scores = []

for class_id in range(1, num_classes):  # Skipping background class (0)
    pred_class = (label == class_id).astype(int)
    target_class = (ground_truth == class_id).astype(int)
    dice_score = calculate_dice_score(pred_class, target_class)
    dice_scores.append(dice_score)
    print(f"Dice score for class {class_id}: {dice_score:.4f}")

# Calculate mean Dice score
mean_dice = sum(dice_scores) / len(dice_scores)
print(f"Mean Dice score: {mean_dice:.4f}")


Dice score for class 1: 0.8638
Dice score for class 2: 0.8659
Mean Dice score: 0.8649
